In [8]:
#Update Prices
# Dependencies

#API Dependencies
import requests
import json
import datetime
apiTimePath = r"..\resources\lastAPICall.txt"
targetURL = "https://www.simcompanies.com/api/v1/market-ticker/2020-04-15T02:15:53.761Z" #Market data URL

#Database Dependencies
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [9]:
#Create table structures (grab from DatabaseMaint notebook if out of date)
class Item(Base):
    __tablename__ = 'item'
    id = Column(Integer, primary_key=True)
    price = Column(Float(2))
    name = Column(String(255))
    kind = Column(Integer)
    updtDtTm = Column(String(20))
    
# Creates a connection to our DB
engine = create_engine("sqlite:///../resources/db/db.sqlite")
conn = engine.connect()

# Create a Session Object to Connect to DB
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [4]:
#Test time of last API call - if less than 6 minutes since last run, don't execute new data call
#Get last time of API call
with open(apiTimePath, 'r+') as apiTimeFile:
    lastApiCallTime = apiTimeFile.readline()

#Convert to datetime format
lastApiCallTime = datetime.datetime.strptime(lastApiCallTime, '%Y-%m-%d %H:%M:%S.%f')

#Compare to current time
curTime = datetime.datetime.now()
timeDiff = (curTime - lastApiCallTime)

#Convert to minutes
minDiff = timeDiff.seconds/60
print(f'Minutes since last refresh: {minDiff:.2f}')

Minutes since last refresh: 25.90


In [5]:
#Handle API Call
if minDiff > 6:
    marketData = requests.get(targetURL).json()
    print("Grabbed market data successfully")
    curTimeStr = str(datetime.datetime.now())
    with open(apiTimePath, 'w') as apiTimeFile:
        apiTimeFile.write(curTimeStr)
        print("Added new time successfully")
    
else:
    print("Fewer than 6 minutes since last API Call - did not execute API Call")

Grabbed market data successfully
Added new time successfully


In [6]:
for i in marketData:
    print(i)

{'kind': 16, 'image': 'images/resources/silicon.png', 'price': 7.6, 'is_up': False}
{'kind': 1, 'image': 'images/resources/power.png', 'price': 0.25, 'is_up': True}
{'kind': 61, 'image': 'images/resources/gloves.png', 'price': 10.3, 'is_up': True}
{'kind': 65, 'image': 'images/resources/sneakers.png', 'price': 12.5, 'is_up': False}
{'kind': 32, 'image': 'images/resources/electronics-research.png', 'price': 410.0, 'is_up': False}
{'kind': 23, 'image': 'images/resources/displays.png', 'price': 119.0, 'is_up': False}
{'kind': 3, 'image': 'images/resources/apples.png', 'price': 1.9, 'is_up': True}
{'kind': 46, 'image': 'images/resources/leather.png', 'price': 13.0, 'is_up': True}
{'kind': 29, 'image': 'images/resources/plant-research.png', 'price': 163.0, 'is_up': False}
{'kind': 111, 'image': 'images/resources/construction-units.png', 'price': 2690.0, 'is_up': True}
{'kind': 2, 'image': 'images/resources/water.png', 'price': 0.308, 'is_up': False}
{'kind': 10, 'image': 'images/resources/c

In [16]:
# Loop through market data
i = 0
for mktItem in marketData:
    #Search for kind in table
    exists = session.query(Item.id).filter_by(kind=mktItem['kind']).scalar() is not None
    
    if exists == False: # and i < 5: #Item doesn't currently exist in table
        #Create array of items to add
        inserts = []
        
        #Extract name from image string
        strTemp = str(mktItem['image']) #image string
        strTemp = strTemp.split('/') #split based on the / character to separate path
        strTemp = strTemp[2].split('.') #then split based on . to separate name from ext
        name = strTemp[0] #0 index now holds our item name
        price = float(mktItem['price'])
        kind = int(mktItem['kind'])
        updtDtTm = str(datetime.datetime.now())
        
        newItem = Item(name=name, price=price, kind=kind, updtDtTm=updtDtTm)
        session.add(newItem)
        #Commit (yikes)
        session.commit()
        print(f'{item} added successfully')
        #i+=1
# Check if kind exists in database - if not, insert
# If does exist, update

    


<__main__.Item object at 0x0000018D66C42808> added successfully
<__main__.Item object at 0x0000018D66C42808> added successfully
<__main__.Item object at 0x0000018D66C42808> added successfully
<__main__.Item object at 0x0000018D66C42808> added successfully
<__main__.Item object at 0x0000018D66C42808> added successfully
<__main__.Item object at 0x0000018D66C42808> added successfully
<__main__.Item object at 0x0000018D66C42808> added successfully
<__main__.Item object at 0x0000018D66C42808> added successfully
<__main__.Item object at 0x0000018D66C42808> added successfully
<__main__.Item object at 0x0000018D66C42808> added successfully
<__main__.Item object at 0x0000018D66C42808> added successfully
<__main__.Item object at 0x0000018D66C42808> added successfully
<__main__.Item object at 0x0000018D66C42808> added successfully
<__main__.Item object at 0x0000018D66C42808> added successfully
<__main__.Item object at 0x0000018D66C42808> added successfully
<__main__.Item object at 0x0000018D66C42

In [18]:
#Test our new data
items = session.query(Item)

for item in items:
    print(f'{item.name} price {item.price}')

xmas-crackers price 29.5
necklace price 1500.0
monitors price 545.0
silicon price 7.6
power price 0.25
gloves price 10.3
sneakers price 12.5
electronics-research price 410.0
displays price 119.0
apples price 1.9
leather price 13.0
plant-research price 163.0
construction-units price 2690.0
water price 0.308
crude-oil price 27.75
automotive-research price 236.0
gold-ore price 33.75
plastic price 10.9
transport price 0.359
steel price 11.7
batteries price 88.0
underwear price 4.3
carbon-fiber price 6.7
grain price 0.54
bauxite price 9.2
diesel price 40.5
methane price 28.25
aluminium price 18.5
eggs price 1.03
oranges price 1.79
reinforced-concrete price 204.0
software price 164.0
seeds price 0.204
combustion-engine price 610.0
ethanol price 23.0
minerals price 11.7
sand price 1.18
tools price 202.0
steak price 11.3
planks price 9.1
rocket-fuel price 44.25
sausages price 3.2
bricks price 2.45
economy-e-car price 3400.0
grapes price 2.0
truck price 5675.0
materials-research price 230.0
ene